In [1]:
import sys
sys.path.insert(1, '../../backend')

from adaptation import AdaptationHandler, create_adapted_module
from execution import execute_test
from stimulus_sheet_reader import get_stimulus_sheet
from solr_parser import parse_solr_response
from solr_query import translate_to_solr_query
from lql.antlr_parser import parse_interface_spec
import pysolr

In [2]:
lql_string = """
Calculator {
    Calculator(int)->None
    log(int, int)->float
    sqrd(int)->str
}
"""
interfaceSpecification = parse_interface_spec(lql_string)

Interface Specification: InterfaceSpecification(className=Calculator, constructors=[MethodSignature(methodName=Calculator, returnType=None, parameterTypes=['int'])], methods=[MethodSignature(methodName=log, returnType=float, parameterTypes=['int', 'int']), MethodSignature(methodName=sqrd, returnType=str, parameterTypes=['int'])])


In [3]:
solr_url = "http://localhost:8983/solr/lasso_quickstart"
solr = pysolr.Solr(solr_url)
solr_query = translate_to_solr_query(interfaceSpecification)
print("QUERY:", solr_query)
results = solr.search(solr_query)
print(f"Found {len(results)} results")

allModulesUnderTest = parse_solr_response(results)
moduleUnderTest = allModulesUnderTest[0]

QUERY: (name:log~0.1 AND (arguments.datatype:('int') AND arguments.datatype:('int'))) OR (name:log~0.1 AND count_positional_args:(2)) OR (name:sqrd~0.1 AND (arguments.datatype:('int'))) OR (name:sqrd~0.1 AND count_positional_args:(1))&group=true&group.field=module
Found 3 results
numpy.lib.scimath
numpy.core.defchararray
Generated 2 ModuleUnderTest objects.


In [4]:
adaptationHandler = AdaptationHandler(interfaceSpecification, moduleUnderTest, excludeClasses=False, useFunctionDefaultValues=False)
adaptationHandler.identifyAdaptations(maxParamPermutationTries=2)
adaptationHandler.visualizeAdaptations()


--------------------
IDENTIFY ADAPTATIONS
--------------------

                  sqrt                            logn
log              None  [NameRtrnConvr, NameRtrnPerm*]
sqrd  [NameRtrnConvr]                            None 



In [5]:
adaptationHandler.generateMappings(onlyKeepTopN=10)

Generated 2 potential mappings.
[log->logn via NameRtrnConvr][sqrd->sqrt via NameRtrnConvr] | distance=6
[log->logn via NameRtrnPerm*][sqrd->sqrt via NameRtrnConvr] | distance=6


In [6]:
(adapted_module, successful_mappings)  = create_adapted_module(adaptationHandler, moduleUnderTest.moduleName, use_constructor_default_values=True)


-----------------------------
TRYING ADAPTATION FOR MAPPING [log->logn via NameRtrnConvr][sqrd->sqrt via NameRtrnConvr] | distance=6.
-----------------------------
Trying to adapt return type of <function logn at 0x7fb31367b550> to float.
Trying to adapt parameter types of <function logn at 0x7fb31367b550> to ['Any', 'Any'].
Created adapted wrapper function.
Adapted name of function <function adapt_function.<locals>.decorator.<locals>.wrapper at 0x7fb315349790> to log.
Trying to adapt return type of <function sqrt at 0x7fb31367b310> to str.
Trying to adapt parameter types of <function sqrt at 0x7fb31367b310> to ['Any'].
Created adapted wrapper function.
Adapted name of function <function adapt_function.<locals>.decorator.<locals>.wrapper at 0x7fb3153499d0> to sqrd.
Successful creation of submodule 0 for this mapping.

-----------------------------
TRYING ADAPTATION FOR MAPPING [log->logn via NameRtrnPerm*][sqrd->sqrt via NameRtrnConvr] | distance=6.
-----------------------------
Tryin

In [7]:
stimulus_sheet = get_stimulus_sheet("calc4_demo.csv")
allSequenceExecutionRecords = execute_test(stimulus_sheet, adapted_module, successful_mappings, interfaceSpecification)
for sequenceExecutionRecord in allSequenceExecutionRecords:
    print(sequenceExecutionRecord)


----------------------
EXECUTE STIMULUS SHEET
----------------------
Module: numpy.lib.scimath
Number of submodules: 2

    output_param method_name  instance_param input_params
1         343.0        sqrd             NaN        [7.0]
2           1.0         log             NaN   [3.0, 2.0]

[log->logn via NameRtrnConvr][sqrd->sqrt via NameRtrnConvr] | distance=6
	sqrd(7.0): 2.6457513110645907, Time: 621 microseconds. Covered lines: 6 in file, 2 in function (100.0% of function lines). Covered arcs: 9 in file, 3 in function.
	log(3.0, 2.0): 0.6309297535714574, Time: 587 microseconds. Covered lines: 7 in file, 3 in function (100.0% of function lines). Covered arcs: 10 in file, 4 in function.

[log->logn via NameRtrnPerm*][sqrd->sqrt via NameRtrnConvr] | distance=6
	sqrd(7.0): 2.6457513110645907, Time: 565 microseconds. Covered lines: 6 in file, 2 in function (100.0% of function lines). Covered arcs: 9 in file, 3 in function.
	log(3.0, 2.0): 1.5849625007211563, Time: 622 microseconds. Co